In [2]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [3]:
 # Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [4]:
 # Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

In [5]:
# Import the CSVs to Pandas DataFrames
file_path = Path("toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

In [6]:
# Panel Visualizations

In [7]:
# Global available data

In [9]:
# Getting the data from the top 10 expensive neighbourhoods
pricey = to_data[['neighbourhood', 'average_house_value']].reset_index().drop(columns='year').groupby('neighbourhood').sum().nlargest(10, 'average_house_value')
pricey.head()


,average_house_value
neighbourhood,
Bridle Path-Sunnybrook-York Mills,6105943
Forest Hill South,4783970
Lawrence Park South,4376111
Rosedale-Moore Park,4374560
St.Andrew-Windfields,3996428


In [11]:
# Calculate the mean number of dwelling types units per year
mean_dwellings =  to_data.groupby("year").sum().drop(columns=["average_house_value", "shelter_costs_owned", "shelter_costs_rented"])
mean_dwellings.head()

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house
year,,,,,,,,
2001,300930,355015,75,90995,52355,23785,116900,3040
2006,266860,379400,165,69430,54690,44095,162850,1335
2011,274940,429220,100,72480,60355,44750,163895,2165
2016,269680,493270,95,71200,61565,48585,165575,2845


In [13]:
# Calculate the average monthly shelter costs for owned and rented dwellings
monthly_shelter_cost = (to_data[["shelter_costs_owned", "shelter_costs_rented"]]).groupby("year").mean().sort_values("year")
monthly_shelter_cost.head()

,shelter_costs_owned,shelter_costs_rented
year,,
2001,846.878571,1085.935714
2006,1316.800000,925.414286
2011,1448.214286,1019.792857
2016,1761.314286,1256.321429


In [14]:
# Panel Visualization Functions

In [29]:
# Define Panel visualization functions
# index free data to call drom  
clean_data = to_data.reset_index()
neigh_map_data = to_data[['neighbourhood', 'average_house_value']].reset_index().drop(columns='year').groupby('neighbourhood').mean()
total_price = pd.concat([df_neighbourhood_locations, neigh_map_data], axis=1, join='inner').reset_index()
avg_house_value = to_data['average_house_value'].groupby(['year']).mean()
values_of_neigh = clean_data[['year','neighbourhood','average_house_value']]
types_of_dwelling = clean_data[["year", "neighbourhood", "single_detached_house", "apartment_five_storeys_plus", "movable_dwelling", "semi_detached_house", "row_house", "duplex", "apartment_five_storeys_less", "other_house"]].set_index("year")
def neighbourhood_map():
    the_map = px.scatter_mapbox(total_price, lat="lat", lon="lon",size="average_house_value",color="neighbourhood", color_continuous_scale=px.colors.cyclical.IceFire, title= "Costs to Live in Toronto",
                                zoom=9, width=600,height=600)
    return the_map
                                
                                
   
    # YOUR CODE HERE!

def create_bar_chart(data, title, xlabel, ylabel, color):
    par = data.plot(kind="bar",title = title, color=color)
    par.set_xlabel(xlabel), par.set_ylabel(ylabel)
    plt.show()
    plt.close()
    
    # YOUR CODE HERE!

def create_line_chart(data, title, xlabel, ylabel, color):
    lar = data.plot(kind="line",title = title, color=color)
    lar.set_xlabel(xlabel), lar.set_ylabel(ylabel)
    plt.show()
    plt.close()
    
    
    # YOUR CODE HERE!

def average_house_value():
    dar = avg_house_value.hvplot(
        x='year', y='average_house_value', title='Average_House_value', 
        xlabel='Years', ylabel='Value', color='purple')
    return dar
    
    
    # YOUR CODE HERE!
def average_value_by_neighbourhood():
    far = value_of_neigh.hvplot(x='year', y='average_house_value', groupby='neighbourhood', color='black', xlabel="Years", ylabel='Value', title='Average_Value_By_neighbourhood')
    return far
    
    # YOUR CODE HERE!

def number_dwelling_types():
    dwel = types_of_dwelling.hvplot(kind='bar',groupby='neighbourhood',  title='Different_Dwellings',
    xlabel="Years", ylabel='Dwelling Type',
    height=500, width=500,rot=90)
    return dwel
    
    
    # YOUR CODE HERE!

def average_house_value_snapshot():
    snap = px.bar(value_of_neigh, x="neighbourhood", y="average_house_value", color="neighbourhood",facet_row="year", 
    height=500,width=500)
    return snap

    # YOUR CODE HERE!

def top_most_expensive_neighbourhoods():
    exp = pricey.plot( kind='bar',title='Most_expensive', color='green')
    exp.set_xlabel('Neighbourhoods')
    exp.set_ylabel('Price')
    plt.show()
    plt.close()

In [27]:
# Panel Dashboard

In [35]:
# Create a Title for the Dashboard
title = "What it Cost to live in the Big City"

# Define a welcome text
welcome = "This dashboard's goal is to provide charts, maps, and interactive visualizations"
welcome_col = pn.Column(
    welcome, neighbourhood_map()
)
costs = pn.Column(
    "Those who own vs those who rent",
    create_line_chart()
    
) 
  
years_display = pn.Column(
    "Year over year",
     create_bar_chart()
)

neigh_col = pn.Column(
    "Average Value",
    average_value_by_neighbourhood()
)
types = pn.Column(
    "Different Typed of Dwelling",
    number_dwelling_types()
)
most_expensive = pn.Column(
    "The most expensive :O", 
    top_most_expensive_neighbourhoods()
    
)
# Create a tab layout for the dashboard

dashboard = pn.Tabs(
    ("Welcome Column", welcome), 
    ("Year over year", years_display),
    ("Costs", costs), 
    ("Avg Value", neigh_col), 
    ("$$$", most_expensive)
)
# Create the main dashboard
dashboard.servable()


KeyError: 'neighbourhood'